# Data Preprocessing - Last Dance

Notebook này tập trung vào bước **chuẩn bị dữ liệu** cho toàn bộ pipeline:

- **Lưu ý:** Chạy notebook `data_cleaning.ipynb` trước để tạo `final_combined_cleaned.csv` và `cleaned_admission.csv`.


In [3]:
import os
import random
import numpy as np
import pandas as pd

# Seed cố định để kết quả reproducible
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

DATA_DIR = 'data'
os.makedirs(DATA_DIR, exist_ok=True)

In [4]:
# Load dữ liệu đã được làm sạch từ data_cleaning.ipynb

# Load các file đã được clean
admission_cleaned = pd.read_csv(os.path.join(DATA_DIR, 'cleaned_admission.csv'))
combined_cleaned = pd.read_csv(os.path.join(DATA_DIR, 'final_combined_cleaned.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

print('cleaned_admission:', admission_cleaned.shape)
print('final_combined_cleaned:', combined_cleaned.shape)
print('test:', test_df.shape)

cleaned_admission: (30217, 6)
final_combined_cleaned: (105726, 15)
test: (16502, 3)


## I. Chuẩn hóa và tách Train / Valid từ final_combined_cleaned

**Theo train_by_year:** Tách Train/Valid theo mask thời gian. **Chỉ chia train** theo năm → train_year_1..6. **Valid không chia theo năm** → lưu 1 file valid.csv.

- **Train:** Từ 2020-2021 đến hết HK1 2023-2024 → chia thành train_year_1..6.
- **Valid:** HK2 2023-2024 → lưu 1 file valid.csv (Dashboard dùng).

In [5]:
# I.1 Chuẩn hóa cột cơ bản

df = combined_cleaned.copy()
df['NAM_TUYENSINH'] = pd.to_numeric(df['NAM_TUYENSINH'], errors='coerce')
df['Nam_Hoc'] = df['Nam_Hoc'].astype(str)
df['_year'] = df['Nam_Hoc'].str.split('-').str[0].astype(float)
df['Hoc_Ky'] = pd.to_numeric(df['Hoc_Ky'], errors='coerce').fillna(1).astype(int)

# Ratio (0-1) dùng làm target trong train/valid
df['Ratio'] = np.nan
mask_tc = (df['TC_DANGKY'].fillna(0) > 0)
df.loc[mask_tc, 'Ratio'] = np.clip(
    df.loc[mask_tc, 'TC_HOANTHANH'].fillna(0) / df.loc[mask_tc, 'TC_DANGKY'], 0, 1
)

df = df.sort_values(['MA_SO_SV', '_year', 'Hoc_Ky']).reset_index(drop=True)
print('Dữ liệu sau chuẩn hóa:', df.shape)

Dữ liệu sau chuẩn hóa: (105726, 17)


In [6]:
# I.2 Tính Year_of_Study
# Logic: Year_of_Study = (năm bắt đầu năm học - năm tuyển sinh + 1)
# Ví dụ: Năm tuyển sinh 2018, record HK1 năm 2020-2021 → năm 3 kỳ 1
#        Tính: 2020 - 2018 + 1 = 3

df['_year_start'] = df['Nam_Hoc'].str.split('-').str[0].astype(float)
df['Year_of_Study'] = (df['_year_start'] - df['NAM_TUYENSINH'] + 1).clip(lower=1).fillna(1).astype(int)

print('Phân bố Year_of_Study:')
print(df['Year_of_Study'].value_counts().sort_index())

# Kiểm tra logic với ví dụ cụ thể
print('\nKiểm tra logic tính Year_of_Study:')
check_example = df[(df['NAM_TUYENSINH'] == 2018) & (df['Nam_Hoc'] == '2020-2021') & (df['Hoc_Ky'] == 1)].head(3)
if len(check_example) > 0:
    print('Ví dụ: Năm tuyển sinh 2018, HK1 năm 2020-2021 → Năm 3 kỳ 1')
    print(check_example[['MA_SO_SV', 'NAM_TUYENSINH', 'Nam_Hoc', 'Hoc_Ky', '_year_start', 'Year_of_Study']])
else:
    # Tìm ví dụ khác
    example = df[df['NAM_TUYENSINH'].notna()].head(5)
    print('Mẫu tính toán Year_of_Study:')
    print(example[['MA_SO_SV', 'NAM_TUYENSINH', 'Nam_Hoc', 'Hoc_Ky', '_year_start', 'Year_of_Study']])

Phân bố Year_of_Study:
Year_of_Study
1    28531
2    26484
3    24304
4    16647
5     8414
6     1346
Name: count, dtype: int64

Kiểm tra logic tính Year_of_Study:
Ví dụ: Năm tuyển sinh 2018, HK1 năm 2020-2021 → Năm 3 kỳ 1
         MA_SO_SV  NAM_TUYENSINH    Nam_Hoc  Hoc_Ky  _year_start  \
20   001565088153           2018  2020-2021       1       2020.0   
81   0034ae77d293           2018  2020-2021       1       2020.0   
107  003fd6214a5c           2018  2020-2021       1       2020.0   

     Year_of_Study  
20               3  
81               3  
107              3  


In [13]:
# I.3 Tách Train / Valid (bước 1: chia theo thời gian)

# Train: 2020-2021 → hết HK1 2023-2024 | Valid: HK2 2023-2024
train_mask = (
    ((df['Nam_Hoc'] >= '2020-2021') & (df['Nam_Hoc'] < '2023-2024')) |
    ((df['Nam_Hoc'] == '2023-2024') & (df['Hoc_Ky'] == 1))
)
valid_mask = (df['Nam_Hoc'] == '2023-2024') & (df['Hoc_Ky'] == 2)

train_df = df[train_mask].copy()
valid_df = df[valid_mask].copy()
print(f'Train: {len(train_df)} rows | Valid: {len(valid_df)} rows')

Train: 90582 rows | Valid: 15144 rows


In [23]:
# I.4 Chia train thành train_year_1..6 (bước 2: từ tập train đã tách)

for year in range(1, 7):
    sub = train_df[train_df['Year_of_Study'] == year].copy()
    if len(sub) == 0:
        continue
    out_path = os.path.join(DATA_DIR, f'train_year_{year}.csv')
    sub.to_csv(out_path, index=False)
    print(f'train_year_{year}.csv: {len(sub)} rows')

train_year_1.csv: 25027 rows
train_year_2.csv: 23154 rows
train_year_3.csv: 20940 rows
train_year_4.csv: 13848 rows
train_year_5.csv: 6822 rows
train_year_6.csv: 791 rows


### I.5 Lưu valid 

In [22]:

valid_path = os.path.join(DATA_DIR, 'valid.csv')
valid_df.to_csv(valid_path, index=False)
print(f'valid.csv: {len(valid_df)} rows')


valid.csv: 15144 rows


## II. Test: chia 2 nhóm (nhập học 2024 + phần còn lại)



In [17]:
# II.1 Merge test với cleaned_admission và tính Year_of_Study

# Merge để lấy NAM_TUYENSINH
test_with_adm = test_df.merge(
    admission_cleaned[['MA_SO_SV', 'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']].drop_duplicates('MA_SO_SV'),
    on='MA_SO_SV',
    how='left'
)

# Parse năm học từ HOC_KY ("HK1 2024-2025" → "2024-2025")
test_with_adm['Nam_Hoc'] = test_with_adm['HOC_KY'].astype(str).str.split().str[-1]
year_start = test_with_adm['Nam_Hoc'].str.split('-').str[0].astype(float)

# Tính Year_of_Study
nam_tuyensinh = pd.to_numeric(test_with_adm['NAM_TUYENSINH'], errors='coerce')
year_of_study_f = (year_start - nam_tuyensinh + 1).clip(lower=1)
mask_valid = year_of_study_f.notna()
test_with_adm.loc[mask_valid, 'Year_of_Study'] = year_of_study_f[mask_valid].astype(int)

# Parse Hoc_Ky từ HOC_KY ("HK1 2024-2025" → 1, "HK2 2024-2025" → 2)
test_with_adm['Hoc_Ky'] = test_with_adm['HOC_KY'].astype(str).str.contains('HK1').astype(int)
test_with_adm['Hoc_Ky'] = test_with_adm['Hoc_Ky'].replace(0, 2)

print('Tổng test sau merge admission:', test_with_adm.shape)
print('Phân bố Year_of_Study trong test:')
print(test_with_adm['Year_of_Study'].value_counts(dropna=False).sort_index())

Tổng test sau merge admission: (16502, 11)
Phân bố Year_of_Study trong test:
Year_of_Study
1.0    4326
2.0    3401
3.0    3263
4.0    3211
5.0    1535
6.0     514
7.0     252
Name: count, dtype: int64


In [24]:
# II.2 Chia test: (1) nhập học 2024  (2) phần còn lại (Y2–Y7)

mask_nhap_hoc_2024 = (test_with_adm['NAM_TUYENSINH'] == 2024)
test_nhap_hoc_2024 = test_with_adm[mask_nhap_hoc_2024].copy()
test_con_lai = test_with_adm[~mask_nhap_hoc_2024].copy()

# Giữ tên cũ cho pipeline: Y1 Sem1 = nhập học 2024; Y2+ = từ test_con_lai
test_Y1_Sem1 = test_nhap_hoc_2024.copy()
test_Y2_plus = test_con_lai.copy()

print(f'test_nhap_hoc_2024 (Y1): {len(test_nhap_hoc_2024)} dòng')
print(f'test_con_lai (Y2–Y7): {len(test_con_lai)} dòng')
if len(test_con_lai) > 0:
    print('Phân bố Year_of_Study trong test_con_lai:')
    print(test_con_lai['Year_of_Study'].value_counts(dropna=False).sort_index())

test_nhap_hoc_2024 (Y1): 4326 dòng
test_con_lai (Y2–Y7): 12176 dòng
Phân bố Year_of_Study trong test_con_lai:
Year_of_Study
2.0    3401
3.0    3263
4.0    3211
5.0    1535
6.0     514
7.0     252
Name: count, dtype: int64


In [25]:
# II.1.1 Kiểm tra các trường hợp đặc biệt trong test

print("=" * 80)
print("KIỂM TRA CÁC TRƯỜNG HỢP ĐẶC BIỆT")
print("=" * 80)

# 1. Kiểm tra sinh viên năm 7
year_7 = test_with_adm[test_with_adm['Year_of_Study'] == 7].copy()
print(f"\n1. Sinh viên năm 7: {len(year_7)} dòng")
if len(year_7) > 0:
    print("   Mẫu 5 dòng đầu:")
    print(year_7[['MA_SO_SV', 'HOC_KY', 'NAM_TUYENSINH', 'Year_of_Study', 'Nam_Hoc']].head())
    print(f"   Phân bố NAM_TUYENSINH: {year_7['NAM_TUYENSINH'].value_counts().sort_index()}")

# 2. Kiểm tra sinh viên có Year_of_Study = NaN
year_nan = test_with_adm[test_with_adm['Year_of_Study'].isna()].copy()
print(f"\n2. Sinh viên có Year_of_Study = NaN: {len(year_nan)} dòng")
if len(year_nan) > 0:
    print("   Chi tiết:")
    print(year_nan[['MA_SO_SV', 'HOC_KY', 'NAM_TUYENSINH', 'Year_of_Study', 'Nam_Hoc']])

# 3. Kiểm tra sinh viên không nhập học từ 2024 (NAM_TUYENSINH != 2024)
not_2024 = test_with_adm[test_with_adm['NAM_TUYENSINH'] != 2024].copy()
print(f"\n3. Sinh viên KHÔNG nhập học từ 2024: {len(not_2024)} dòng")
if len(not_2024) > 0:
    print(f"   Phân bố NAM_TUYENSINH: {not_2024['NAM_TUYENSINH'].value_counts().sort_index()}")
    print(f"   Phân bố Year_of_Study: {not_2024['Year_of_Study'].value_counts(dropna=False).sort_index()}")
    print("   Mẫu 10 dòng đầu:")
    print(not_2024[['MA_SO_SV', 'HOC_KY', 'NAM_TUYENSINH', 'Year_of_Study', 'Nam_Hoc', 'Hoc_Ky']].head(10))

# 4. Kiểm tra sinh viên nhập học 2024
year_2024 = test_with_adm[test_with_adm['NAM_TUYENSINH'] == 2024].copy()
print(f"\n4. Sinh viên nhập học 2024: {len(year_2024)} dòng")
if len(year_2024) > 0:
    print(f"   Phân bố Year_of_Study: {year_2024['Year_of_Study'].value_counts(dropna=False).sort_index()}")
    print(f"   Phân bố Hoc_Ky: {year_2024['Hoc_Ky'].value_counts().sort_index()}")
    print("   Mẫu 5 dòng đầu:")
    print(year_2024[['MA_SO_SV', 'HOC_KY', 'NAM_TUYENSINH', 'Year_of_Study', 'Nam_Hoc', 'Hoc_Ky']].head())

# 5. Kiểm tra logic tính Year_of_Study cho sinh viên nhập học 2024
print(f"\n5. Kiểm tra logic tính Year_of_Study cho sinh viên nhập học 2024:")
if len(year_2024) > 0:
    year_2024_check = year_2024.copy()
    year_2024_check['year_start_calc'] = year_2024_check['Nam_Hoc'].str.split('-').str[0].astype(float)
    year_2024_check['calc_year'] = (year_2024_check['year_start_calc'] - year_2024_check['NAM_TUYENSINH'] + 1)
    print("   Mẫu 10 dòng với tính toán:")
    print(year_2024_check[['MA_SO_SV', 'HOC_KY', 'NAM_TUYENSINH', 'year_start_calc', 'calc_year', 'Year_of_Study', 'Hoc_Ky']].head(10))

print("\n" + "=" * 80)

KIỂM TRA CÁC TRƯỜNG HỢP ĐẶC BIỆT

1. Sinh viên năm 7: 252 dòng
   Mẫu 5 dòng đầu:
          MA_SO_SV         HOC_KY  NAM_TUYENSINH  Year_of_Study    Nam_Hoc
1     6c8a97d22131  HK1 2024-2025           2018            7.0  2024-2025
3     438aff5ef524  HK1 2024-2025           2018            7.0  2024-2025
5     b87bd14ae979  HK1 2024-2025           2018            7.0  2024-2025
291   6d7e46e1c3fb  HK1 2024-2025           2018            7.0  2024-2025
1062  db5d290e6a89  HK1 2024-2025           2018            7.0  2024-2025
   Phân bố NAM_TUYENSINH: NAM_TUYENSINH
2018    252
Name: count, dtype: int64

2. Sinh viên có Year_of_Study = NaN: 0 dòng

3. Sinh viên KHÔNG nhập học từ 2024: 12176 dòng
   Phân bố NAM_TUYENSINH: NAM_TUYENSINH
2018     252
2019     514
2020    1535
2021    3211
2022    3263
2023    3401
Name: count, dtype: int64
   Phân bố Year_of_Study: Year_of_Study
2.0    3401
3.0    3263
4.0    3211
5.0    1535
6.0     514
7.0     252
Name: count, dtype: int64
   Mẫu 10 dòng

In [27]:
# II.3 Lưu test: 2 nhóm (nhập học 2024 + phần còn lại) + test_Y1_Sem1 cho Y1; không lưu test_Y2..Y7 riêng

path_2024 = os.path.join(DATA_DIR, 'test_nhap_hoc_2024.csv')
path_con_lai = os.path.join(DATA_DIR, 'test_con_lai.csv')
test_nhap_hoc_2024.to_csv(path_2024, index=False)
test_con_lai.to_csv(path_con_lai, index=False)
print(f'Đã lưu {path_2024}: {len(test_nhap_hoc_2024)} dòng')
print(f'Đã lưu {path_con_lai}: {len(test_con_lai)} dòng')

# Y1: lưu test_Y1_Sem1.csv để feature_engineering dùng
test_Y1_Sem1.to_csv(os.path.join(DATA_DIR, 'test_Y1_Sem1.csv'), index=False)
print(f'test_Y1_Sem1.csv: {len(test_Y1_Sem1)} dòng')

Đã lưu data/test_nhap_hoc_2024.csv: 4326 dòng
Đã lưu data/test_con_lai.csv: 12176 dòng
test_Y1_Sem1.csv: 4326 dòng
